## Introduction

In this lab assessment, you'll practice your knowledge of JOIN statements and subqueries, using various types of joins and various methods for specifying the links between them. One of the main benefits of using a relational database is the table relations that define them which allow you to access and connect data together via shared columns. By writing more advanced SQL queries that utilize joins and subqueries you can provide a deeper and more granular level of analysis and data retrieval.

This assessment will continue looking at the familiar Northwind database that contains customer relationship management (CRM) data as well as employee and product data. You will take a deeper dive into this database in order to accomplish more advanced SQL queries that require you to access data from multiple tables at once. 

Imagine that you are working in an analyst role for the sales rep team. They have collaborated with the customer relations and the product teams to take a comprehensive look at the employee to customer pipeline in an attempt to find areas of improvement and potential growth. You have been asked to provide some specific data and statistics regarding this project.

## Learning Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Choose and perform whichever type of join is best for retrieving desired data
* Write subqueries to decompose complex queries

## Database

The database will be the customer relationship management (CRM) database, which has the following ERD.

![Database-Schema.png](ERD.png)

### Connect to the database

In the cell below we have provided the code to import both pandas and sqlite3 as well as define and create the connection to the database you will use. Also displayed is the schema and table names from the database. Use this information in conjunction with the ERD image above to assist in creating your SQL Queries.

Major Hint: Look for the shared columns across tables you need to 'join' together.

In [1]:
# CodeGrade step0
# Run this cell without changes

# SQL Library and Pandas Library
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('data.sqlite')

pd.read_sql("""SELECT * FROM sqlite_master""", conn)

,type,name,tbl_name,rootpage,sql
0,table,orderdetails,orderdetails,2,"CREATE TABLE `orderdetails` (`orderNumber`, `p..."
1,table,payments,payments,28,"CREATE TABLE `payments` (`customerNumber`, `ch..."
2,table,offices,offices,32,"CREATE TABLE `offices` (`officeCode`, `city`, ..."
3,table,customers,customers,33,"CREATE TABLE `customers` (`customerNumber`, `c..."
4,table,orders,orders,38,"CREATE TABLE `orders` (`orderNumber`, `orderDa..."
5,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
6,table,products,products,47,"CREATE TABLE `products` (`productCode`, `produ..."
7,table,employees,employees,56,"CREATE TABLE `employees` (`employeeNumber`, `l..."


## Part 1: Join and Filter

### Step 1

The company would like to let Boston employees go remote but need to know more information about who is working in that office. Return the first and last names and the job titles for all employees in Boston.

In [2]:
# CodeGrade step1
# Replace None with your code
df_boston = pd.read_sql(
"""
select e.lastName, e.firstName, e.jobTitle
from employees e
join offices using(officeCode)
where city = 'Boston';
""", conn)
df_boston

,lastName,firstName,jobTitle
0,Firrelli,Julie,Sales Rep
1,Patterson,Steve,Sales Rep


### Step 2

Recent downsizing and employee attrition have caused some mixups in office tracking and the company is worried they are supporting a 'ghost' location. Are there any offices that have zero employees?

In [3]:
# CodeGrade step2
# Replace None with your code
df_zero_emp = pd.read_sql(
"""
select o.officeCode,o.city, count(e.employeeNumber) as no_emp
from employees e
join offices o using(officeCode)
group by o.officeCode
order by no_emp desc;
""", conn)
df_zero_emp

,officeCode,city,no_emp
0,1,San Francisco,6
1,4,Paris,5
2,6,Sydney,4
3,7,London,2
4,5,Tokyo,2
5,3,NYC,2
6,2,Boston,2


## Part 2: Type of Join

### Step 3

As a part of this larger analysis project the HR department is taking the time to audit employee records to make sure nothing is out of place and have asked you to produce a report of all employees. Return the employees first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [4]:
# CodeGrade step3
# Replace None with your code
df_employee = pd.read_sql(
"""
select e.firstName, e.lastName, o.city, o.state
from employees e
left join offices o using(officeCode)
order by e.firstName, e.lastName;
""", conn)
df_employee

,firstName,lastName,city,state
0,Andy,Fixter,Sydney,
1,Anthony,Bow,San Francisco,CA
2,Barry,Jones,London,
3,Diane,Murphy,San Francisco,CA
4,Foon Yue,Tseng,NYC,NY
5,George,Vanauf,NYC,NY
6,Gerard,Bondur,Paris,
7,Gerard,Hernandez,Paris,
8,Jeff,Firrelli,San Francisco,CA
9,Julie,Firrelli,Boston,MA


### Step 4
The customer management and sales rep team know that they have several 'customers' in the system that have not placed any orders. They want to reach out to these customers with updated product catalogs to try and get them to place initial orders. Return all of the customer's contact information (first name, last name, and phone number) as well as their sales rep's employee number for any customer that has not placed an order. Sort the results alphabetically based on the contact's last name

There are several approaches you could take here, including a left join and filtering on null values or using a subquery to filter out customers who do have orders. In total there are 24 customers who have not placed an order.

In [5]:
# CodeGrade step4
# Replace None with your code
df_contacts = pd.read_sql(
"""
select c.contactFirstName, c.contactLastName, c.phone, c.salesRepEmployeeNumber
from customers c
left join orders o using(customerNumber)
where o.orderNumber is NULL;
""", conn)
df_contacts

,contactFirstName,contactLastName,phone,salesRepEmployeeNumber
0,Zbyszek,Piestrzeniewicz,(26) 642-7555,
1,Keith,Franco,2035557845,1286
2,Isabel,de Castro,(1) 356-5555,
3,Brydey,Walker,+612 9411 1555,
4,Horst,Kloss,0372-555188,
5,Alejandra,Camino,(91) 745 6555,
6,Renate,Messner,069-0555984,
7,Peter,Franken,089-0877555,
8,Ed,Harrison,+41 26 425 50 01,
9,Bradley,Schuyler,+31 20 491 9555,


In [6]:
print(f'Total Customers w/o Orders (count rows): {len(df_contacts)}')

Total Customers w/o Orders (count rows): 24


## Part 3: Built-in Function

### Step 5

The accounting team is auditing their figures and wants to make sure all customer payments are in alignment, they have asked you to produce a report of all the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. They have asked that these results be sorted in descending order by the payment amount.

Hint: A member of their team mentioned that they are not sure the 'amount' column is being stored as the right datatype so keep this in mind when sorting.

In [7]:
# CodeGrade step5
# Replace None with your code
df_payment = pd.read_sql(
"""
select c.contactFirstName, c.contactLastName, p.amount, p.paymentDate
from customers c
join payments p using(customerNumber)
order by p.amount;
""", conn)
df_payment

,contactFirstName,contactLastName,amount,paymentDate
0,Susan,Nelson,101244.59,2005-03-05
1,Dorothy,Young,10223.83,2003-01-16
2,Roland,Keitel,10549.01,2003-01-28
3,Eric,Natividad,105743.00,2003-12-26
4,Susan,Nelson,11044.30,2003-04-11
...,...,...,...,...
268,Roland,Mendel,8807.12,2005-05-03
269,Sean,Clenahan,9415.13,2004-07-28
270,Leslie,Taylor,9658.74,2004-12-06
271,Ben,Calaghan,9821.32,2003-10-17


## Part 4: Joining and Grouping

### Step 6

The sales rep team has noticed several key team members that stand out as having trustworthy business relations with their customers, reflected by high credit limits indicating more potential for orders. The team wants you to identify these 4 individuals. Return the employee number, first name, last name, and number of customers for employees whose customers have an average credit limit over 90k. Sort by number of customers from high to low.

In [8]:
# CodeGrade step6
# Replace None with your code
df_credit = pd.read_sql(
"""
select e.employeeNumber, e.firstName, e.lastName, count(c.customerNumber) as no_cust
from employees e
join customers c
on e.employeeNumber = c.salesRepEmployeeNumber
group by e.employeeNumber
having avg(c.creditLimit) > 90000
""", conn)
df_credit

,employeeNumber,firstName,lastName,no_cust
0,1165,Leslie,Jennings,6
1,1370,Gerard,Hernandez,7
2,1501,Larry,Bott,8
3,1612,Peter,Marsh,5


### Step 7

The product team is looking to create new model kits and wants to know which current products are selling the most in order to get an idea of what is popular. Return the product name and count the number of orders for each product as a column named 'numorders'. Also return a new column, 'totalunits', that sums up the total quantity of product sold (use the quantityOrdered column). Sort the results by the totalunits column, highest to lowest, to showcase the top selling products.

In [9]:
# CodeGrade step7
# Replace None with your code
df_product_sold = pd.read_sql(
"""
select p.productName, count(o.orderNumber) as numorders, sum(o.quantityOrdered) as totalunits
from products p
join orderdetails o using(productCode)
group by p.productCode
order by totalunits desc;
""", conn)
df_product_sold

,productName,numorders,totalunits
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,25,855
105,1911 Ford Town Car,25,832
106,1936 Mercedes Benz 500k Roadster,25,824
107,1970 Chevy Chevelle SS 454,25,803


## Part 5: Multiple Joins

### Step 8

As a follow-up to the above question, the product team also wants to know how many different customers ordered each product to get an idea of market reach. Return the product name, code, and the total number of customers who have ordered each product, aliased as 'numpurchasers'. Sort the results by the highest  number of purchasers.

Hint: You might need to join more than 2 tables. Use DISTINCT to return unique/different values.

In [10]:
# CodeGrade step8
# Replace None with your code
df_total_customers = pd.read_sql(
"""
select p.productName, p.productCode, count(distinct(o.customerNumber)) as numpurchasers
from products p
join orderdetails od using(productCode)
join orders o using(orderNumber)
group by p.productCode
order by numpurchasers desc;
""", conn)
df_total_customers

,productName,productCode,numpurchasers
0,1992 Ferrari 360 Spider red,S18_3232,40
1,Boeing X-32A JSF,S72_1253,27
2,1934 Ford V8 Coupe,S18_2957,27
3,1972 Alfa Romeo GTA,S10_4757,27
4,1952 Alpine Renault 1300,S10_1949,27
...,...,...,...
104,1958 Chevy Corvette Limited Edition,S24_2840,19
105,2002 Chevy Corvette,S24_3432,18
106,1969 Chevrolet Camaro Z28,S24_3191,18
107,1952 Citroen-15CV,S24_2887,18


### Step 9

The custom relations team is worried they are not staffing locations properly to account for customer volume. They want to know how many customers there are per office. Return the count as a column named 'n_customers'. Also return the office code and city.

In [11]:
# CodeGrade step9
# Replace None with your code
df_customers = pd.read_sql(
"""
select count(c.customerNumber) as n_customers, o.officeCode, o.city
from offices o
join employees e using(officeCode)
join customers c
on e.employeeNumber = c.salesRepEmployeeNumber
group by o.officeCode;
""", conn)
df_customers

,n_customers,officeCode,city
0,12,1,San Francisco
1,12,2,Boston
2,15,3,NYC
3,29,4,Paris
4,5,5,Tokyo
5,10,6,Sydney
6,17,7,London


## Part 6: Subquery

### Step 10

Having looked at the results from above, the product team is curious to dig into the underperforming products. They want to ask members of the team who have sold these products about what kind of messaging was successful in getting a customer to buy these specific products. Using a subquery or common table expression (CTE), select the employee number, first name, last name, city of the office, and the office code for employees who sold products that have been ordered by fewer than 20 customers.

Hint: Start with the subquery, find all the products that have been ordered by 19 or less customers, consider adapting one of your previous queries.

In [12]:
# CodeGrade step10
# Replace None with your code
df_under_20 = None

### Close the connection

In [13]:
# Run this cell without changes

#conn.close()